In [ ]:
import numpy as np
import pickle
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import seaborn as sns
import os
import time
import sys
import pandas as pd
import time
import matplotlib.pyplot as plt
# import torch.optim as optim
from FwFM_Network import FwFM_ForCELoss, FwFM_Logits
from Utils import CreateDataset, runCodeGreedy, retBidsAndCamps2nd, retBidsAndCampsGreedy, spendAndRevenuePerAdvertiser, createTrValTestData
from Utils_res import get_total_measures_per_run_camp, create_df_from_dict, conv_to_dict_w_arrays, conv_to_dict_w_arrays
print('Load libraries')

## Read Data and fixed some parameters

In [ ]:
current_directory = os.getcwd()
pathToReadData = os.path.join(current_directory, 'DataForPred/')

mult_for_cost = 100
# seed to use does not affect the result
seed_to_use = 17
num_of_indxs_to_use = 128

# We do not need q here. 
X_Train, X_Val, X_Test, extra_train, extra_val, extra_test, target , _, lbs, \
    camps_identifiers, num_advs, avg_cost_per_conv = \
    createTrValTestData(pathToReadData, mult_for_cost, seed_to_use, num_of_indxs_to_use)

np.random.seed(12369)
possible_seeds = np.random.choice(np.arange(1000000) + 1, size = 100, replace = False)


### Number of test points, number of iterations run, and maxBudgets

In [ ]:
totPoints = X_Test.size()[0]
numIte = int(totPoints/num_of_indxs_to_use)
maxBudgets = (target * numIte).numpy()

## Get Results for Dual Method

In [ ]:
index_step_or_bid = 0
num_of_sims = 100
path_raw_results = "path_to_stored_raw_results/"
pathToReadResults_2nd = os.path.join(path_raw_results, 'Results2nd/')

In [ ]:
possible_step_sizes = [1.0, 0.5, 0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001]
profit_all_2nd, spend_all_2nd, vio_all_2nd, ites_budget_all_2nd = {}, {}, {}, {}
for index_step in range(len(possible_step_sizes)):
    start_time = time.time()
    profit_aux, spend_aux, vio_all_aux, ites_budgets_aux = \
        get_total_measures_per_run_camp(index_step, num_of_sims, numIte, num_advs, maxBudgets, pathToReadResults_2nd)
    profit_all_2nd[possible_step_sizes[index_step]] = profit_aux
    spend_all_2nd[possible_step_sizes[index_step]] = spend_aux
    vio_all_2nd[possible_step_sizes[index_step]] = vio_all_aux
    ites_budget_all_2nd[possible_step_sizes[index_step]] = ites_budgets_aux
    print('Finishing getting aggregated data for step_size ' + str(possible_step_sizes[index_step]), end = ', ')
    print('took ' + str(time.time() - start_time) + ' secs.')

## Get Results Greedy Method

In [ ]:
pathToReadResults_Gr = os.path.join(path_raw_results, 'ResultsGr/')
num_of_sims_Gr = 100

In [ ]:
possible_bid_mult= [0.5 + 0.05 * i for i in range(21)]
possible_bid_mult.extend([0.25 + 0.05*i for i in range(5)])
index_to_use = [i for i in range(11)]
index_to_use.extend([i for i in range(21, 26)])
profit_all_Gr, spend_all_Gr, vio_all_Gr, ites_budget_all_Gr = {}, {}, {}, {}
for index_bid in index_to_use:
    start_time = time.time()
    profit_aux, spend_aux, vio_all_aux, ites_budgets_aux = \
        get_total_measures_per_run_camp(index_bid, num_of_sims_Gr, numIte, num_advs, maxBudgets, pathToReadResults_Gr)
    profit_all_Gr[possible_bid_mult[index_bid]] = profit_aux
    spend_all_Gr[possible_bid_mult[index_bid]] = spend_aux
    vio_all_Gr[possible_bid_mult[index_bid]] = vio_all_aux
    ites_budget_all_Gr[possible_bid_mult[index_bid]] = ites_budgets_aux
    print('Finishing getting aggregated data for bid multiplier ' + str(possible_bid_mult[index_bid]), end = ', ')
    print('took ' + str(time.time() - start_time) + ' secs.')

# Get budget utilization per campaign

In [ ]:
budget_util_all_2nd, budget_util_all_greedy = {}, {}

for key_name in list(spend_all_2nd.keys()):
    budget_util_all_2nd[key_name] = np.zeros((num_of_sims, num_advs))
    for i in range(num_of_sims):
        budget_util_all_2nd[key_name][i,:] =  spend_all_2nd[key_name][i,:]/maxBudgets

for key_name in list(spend_all_Gr.keys()):
    budget_util_all_greedy[key_name] = np.zeros((num_of_sims_Gr, num_advs))
    for i in range(num_of_sims_Gr):
        budget_util_all_greedy[key_name][i,:] =  spend_all_Gr[key_name][i,:]/maxBudgets

In [ ]:
budget_util_avg_2nd, budget_util_avg_Gr = {}, {}
for key_name in list(profit_all_2nd.keys()):
    budget_util_avg_2nd[key_name] =  np.average(budget_util_all_2nd[key_name], axis=0)

for key_name in list(profit_all_Gr.keys()):
    budget_util_avg_Gr[key_name] =  np.average(budget_util_all_greedy[key_name], axis=0)

## Process data for the average exceed time

In [ ]:
ite_exceed_dict_2nd =  conv_to_dict_w_arrays(ites_budget_all_2nd, -1, numIte)
ite_exceed_dict_Gr =  conv_to_dict_w_arrays(ites_budget_all_Gr, -1, numIte)

In [ ]:
profit_tot_2nd, spend_tot_2nd, vio_avg_2nd = {}, {}, {}
profit_tot_Gr, spend_tot_Gr, vio_avg_Gr = {}, {}, {}

In [ ]:
for key_name in list(profit_all_2nd.keys()):
    profit_tot_2nd[key_name] =  np.sum(profit_all_2nd[key_name], axis=1)
    spend_tot_2nd[key_name] =  np.sum(spend_all_2nd[key_name], axis=1)
    vio_avg_2nd[key_name] =  np.average(vio_all_2nd[key_name], axis=0)

for key_name in list(profit_all_Gr.keys()):
    profit_tot_Gr[key_name] =  np.sum(profit_all_Gr[key_name], axis=1)
    spend_tot_Gr[key_name] =  np.sum(spend_all_Gr[key_name], axis=1)
    vio_avg_Gr[key_name] =  np.average(vio_all_Gr[key_name], axis=0)

# Just to print, check the average values on profit_tot_2nd and profit_tot_Gr

In [ ]:
for key_name in profit_tot_2nd.keys():
    print(str(key_name) + ', profit ' + str(np.average(profit_tot_2nd[key_name])) + ', median ' + str(np.median(profit_tot_2nd[key_name])))
print()
for key_name in profit_tot_Gr.keys():
    print(str(key_name) + ', profit ' + str(np.average(profit_tot_Gr[key_name])) + ', median ' + str(np.median(profit_tot_Gr[key_name])))

## Get dataframes for Profit, Spend , First Exceed

In [ ]:
pd_tot_profit_2nd = create_df_from_dict(profit_tot_2nd, 'step_size', 'total_profit')
pd_tot_profit_Gr = create_df_from_dict(profit_tot_Gr, 'bid_multiplier', 'total_profit')
pd_tot_spend_2nd = create_df_from_dict(spend_tot_2nd, 'step_size', 'total_spend')
pd_tot_spend_Gr = create_df_from_dict(spend_tot_Gr, 'bid_multiplier', 'total_spend')
pd_vio_camp_2nd = create_df_from_dict(vio_avg_2nd, 'step_size', 'vio_per_adv')
pd_vio_camp_Gr = create_df_from_dict(vio_avg_Gr, 'step_size', 'vio_per_adv')
pd_ite_vio_2nd = create_df_from_dict(ite_exceed_dict_2nd, 'step_size', 'first_vio')
pd_ite_vio_Gr = create_df_from_dict(ite_exceed_dict_Gr, 'bid_multiplier', 'first_vio')
pd_budget_util_2nd = create_df_from_dict(budget_util_avg_2nd, 'step_size', 'budget_utilization')
pd_budget_util_Gr = create_df_from_dict(budget_util_avg_Gr, 'bid_multiplier', 'budget_utilization')

In [ ]:
len(vio_avg_2nd[1.0])

# Results Dual Method

In [ ]:
sns.boxplot(x="step_size", y="total_profit", data=pd_tot_profit_2nd);

In [ ]:
sns.boxplot(x="step_size", y="total_spend", data=pd_tot_spend_2nd);

In [ ]:
sns.boxplot(x="step_size", y="first_vio", data=pd_ite_vio_2nd);

In [ ]:
sns.boxplot(x="step_size", y="budget_utilization", data=pd_budget_util_2nd);

In [ ]:
sns.boxplot(x="step_size", y="vio_per_adv", data=pd_vio_camp_2nd);

# Results Greedy Policy

In [ ]:
sns.boxplot(x="bid_multiplier", y="total_profit", data=pd_tot_profit_Gr);

In [ ]:
sns.boxplot(x="bid_multiplier", y="total_spend", data=pd_tot_spend_Gr);

In [ ]:
sns.boxplot(x="bid_multiplier", y="first_vio", data=pd_ite_vio_Gr);

In [ ]:
sns.boxplot(x="bid_multiplier", y="budget_utilization", data=pd_budget_util_Gr);

In [ ]:
sns.boxplot(x="step_size", y="vio_per_adv", data=pd_vio_camp_Gr);

## Results to show

In [ ]:
best_profit_2nd = pd_tot_profit_2nd[pd_tot_profit_2nd['step_size'] == 0.05]
best_profit_Gr = pd_tot_profit_Gr[pd_tot_profit_Gr['bid_multiplier'] == 0.25]

best_vio_2nd = pd_vio_camp_2nd[pd_vio_camp_2nd['step_size'] == 0.05]
best_vio_Gr = pd_vio_camp_Gr[pd_vio_camp_Gr['step_size'] == 0.25]

best_bu_2nd = pd_budget_util_2nd[pd_budget_util_2nd['step_size'] == 0.05]
best_bu_Gr = pd_budget_util_Gr[pd_budget_util_Gr['bid_multiplier'] == 0.25]

total_profit = np.array(list(best_profit_2nd['total_profit'])+ list(best_profit_Gr['total_profit']))
array_vio_camps = np.array(list(best_vio_2nd['vio_per_adv'])+ list(best_vio_Gr['vio_per_adv']))
array_bu_camps = np.array(list(best_bu_2nd['budget_utilization'])+ list(best_bu_Gr['budget_utilization']))

method_names_profit = ['Our Method' for i in range(100)] + ['Heuristic' for i in range(100)]
method_names_vio = ['Our Method' for i in range(130)] + ['Heuristic' for i in range(130)]


In [ ]:
df_profit_for_show = pd.DataFrame({'Total Profit': total_profit, 'Method Name': method_names_profit}) 
df_vio_for_show = pd.DataFrame({'Iteration Budget Depleted': array_vio_camps, 'Method Name': method_names_vio}) 
df_bu_for_show = pd.DataFrame({'Budget Utilization': array_bu_camps, 'Method Name': method_names_vio}) 

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(10,4))

sns.boxplot(y="Method Name", x="Total Profit", data=df_profit_for_show, ax=ax1)
sns.boxplot(y="Method Name", x="Iteration Budget Depleted", data=df_vio_for_show, ax=ax3)
sns.boxplot(y="Method Name", x="Budget Utilization", data=df_bu_for_show, ax=ax2)

ax1.set_yticklabels(['Our Method', 'Heuristic'], rotation = 90, fontsize = 14)
ax1.set(ylabel = "")
ax1.xaxis.label.set_size(14)
ax2.label_outer()
ax2.xaxis.label.set_size(14)
ax3.label_outer()
ax3.xaxis.label.set_size(14)


SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

fig.tight_layout()

plt.rc('font', size=BIGGER_SIZE)  
plt.savefig('box_plots_online_2nd.png')
plt.show()


In [ ]:
plt.style.use('seaborn-whitegrid')
plt.xscale('log')
plt.xlabel("Proportional budget w.r.t. maximum (log. scale) ")
plt.ylabel("Average depletion time")
plt.rc('axes', labelsize=16)
plt.scatter(maxBudgets/np.max(maxBudgets), np.array(best_vio_2nd['vio_per_adv']), marker='o');
plt.tight_layout()
plt.savefig('TimeVsRelativeBudget.png')